In [0]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=73dfbdb6e7c766d43e119631a7d49503ac2e3958440f20fb19cc0405a8f69bc1
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

trainfile = r'/gdrive/My Drive/CIS 508/Assignment/HW3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS 508/Assignment/HW3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [0]:
#Copy Train data excluding target
X_train = train_data.iloc[:, :-1]
X_test = test_data.iloc[:, :-1]

#Select just Target Column
y_train = train_data.iloc[:, -1]
# y_test = test_data.iloc[:, -1]

print(X_train.shape)
print(X_test.head()) 

print(y_train.shape)
# print(y_test.head()) 

(65000, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]
(65000,)


In [0]:
#CONSTRUCT DEFAULT DECISION TREE
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)

prediction = pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])
pd.concat([X_test['QuoteNumber'],prediction],axis=1).to_csv(r'/gdrive/My Drive/CIS 508/Assignment/HW3/DTree_NoParameter.csv', index = None)

In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)


prediction = pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])
pd.concat([X_test['QuoteNumber'],prediction],axis=1).to_csv(r'/gdrive/My Drive/CIS 508/Assignment/HW3/DTree_HyperParameter.csv', index = None)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
# print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
# print("Confusion Matrix after hypertuning for Decision Tree")
# print(confusion_matrix(y_test,clf_predict))
# print("=== Classification Report ===")
# print(classification_report(y_test,clf_predict))

# run cross-validation on best hyperparameters, get auc score
# clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
# print("=== All AUC Scores ===")
# print(clf_cv_score)
# print('\n')
# print("=== Mean AUC Score ===")
# print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 90, 'max_depth': 7}


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)

prediction = pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])
pd.concat([X_test['QuoteNumber'],prediction],axis=1).to_csv(r'/gdrive/My Drive/CIS 508/Assignment/HW3/RForest_NoParameter.csv', index = None)

In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

prediction = pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])
pd.concat([X_test['QuoteNumber'],prediction],axis=1).to_csv(r'/gdrive/My Drive/CIS 508/Assignment/HW3/RForest_HyperParameter.csv', index = None)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 90, 'min_samples_split': 10, 'max_depth': 19}
=== All AUC Scores ===
[0.95023951 0.94992596 0.94049466 0.95305945 0.94731024 0.95414669
 0.95184261 0.94512478 0.95346743 0.940428  ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9486039341492388


In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

{'n_estimators': 30, 'learning_rate': 0.1}


In [0]:
#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

=== All AUC Scores ===
[0.94933187 0.94452689 0.94074907 0.948058   0.94507815 0.95126505
 0.95335972 0.93951955 0.94990872 0.93941921]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.9461216232198224


In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.92154329]
    fold  1:  [0.92037413]
    fold  2:  [0.92313373]
    fold  3:  [0.92190289]
    ----
    MEAN:     [0.92173851] + [0.00098404]
    FULL:     [0.92173846]

model  1:     [RandomForestClassifier]
    fold  0:  [0.89280660]
    fold  1:  [0.88929912]
    fold  2:  [0.89783987]
    fold  3:  [0.89014709]
    ----
    MEAN:     [0.89252317] + [0.00333121]
    FULL:     [0.89252308]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.88253031]
    fold  1:  [0.88308412]
    fold  2:  [0.88325435]
    fold  3:  [0.88497754]
    ----
    MEAN:     [0.88346158] + [0.00091526]
    FULL:     [0.88346154]



In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_train)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(y_test,y_pred))
print("=== Classification Report ===")
print(classification_report(y_test,y_pred))

NameError: ignored

In [0]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.946377,0.053623
1,0.946377,0.053623
2,0.946377,0.053623
3,0.946377,0.053623
4,0.946377,0.053623
